# CONNECT THE BOARD

In [ ]:
%serialconnect to --port="COM5" --baud=115200

In [ ]:
f = open("main.py", "r")
print(f.read())

In [ ]:
%sendtofile lib/collector.py --source lib/collector.py
%sendtofile lib/peripherals.py --source lib/peripherals.py
%sendtofile lib/signal.py --source lib/signal.py

In [8]:
%sendtofile main.py --source main.py

Sent 73 lines (1764 bytes) to main.py.


# COLLECTOR METHOD

In [ ]:
import gc
from micropython import alloc_emergency_exception_buf

from lib.requests import MicroWebCli as requests
import ujson as json

from lib.utils import connect_wifi, load_env_vars

from ulab import numpy as np
import utime as time
from lib.collector import Collector 
from lib.runner import Runner 

In [ ]:
collector = Collector(buffer_size=256, preprocess=True, ds=True)

SPI_params= {
    "spi_num": 2,
    "sck": 18,
    "mosi": 23,
    "miso": 19,
    "output_amp_gain": 255,
}

collector.setup(spi_params=SPI_params)

# ssid = 'TP-Link_AP_4C04'
# password = '63525465'
# connect_wifi(ssid, password)
time.sleep(4)

collector.run()

time.sleep(10)

while True:
    time.sleep(1)
    bt = time.time()
    data = collector.read_output_buffer()
    et = time.time()
    print(et-bt,data)
#     toSend = {"raw_data":data}
#     print(toSend)
#     requests.JSONRequest("http://192.168.0.13:5001/collect", toSend)
#     # requests.JSONRequest("http://172.20.10.2:5001/collect", toSend)
#     del data
#     del toSend
#     # requests.GETRequest("http://172.20.10.2:5001/save")
#     requests.GETRequest("http://192.168.0.13:5001/save")

In [ ]:
collector.stop()

In [ ]:
runner = Runner('CCA', buffer_size=256)
runner.setup()

# ssid = 'VM0857951'
# password = 'cchk7mrYgrzq'
# connect_wifi(ssid, password)

time.sleep(4)
runner.run()
while True:
    time.sleep(1)
    data = runner.read_output_buffer()
    print(data)

In [ ]:
runner.stop()

# DIRECTLY READ ADC, SKIP EVERYTHING ELSE

In [ ]:
%serialconnect to --port="COM5" --baud=115200

In [ ]:
import machine
from machine import Pin
from machine import ADC, SPI
import time
from machine import Timer
import gc
from machine import freq
from lib.utils import connect_wifi
from lib.requests import MicroWebCli as requests
import ujson as json

freq(240000000)
adc = ADC(Pin(33))
adc.atten(machine.ADC.ATTN_11DB)
adc.width(machine.ADC.WIDTH_12BIT)
adcread = adc.read()

##################################################################################

DEFAULT_SPI_PARAMS = {
    "spi_num": 2,
    "sck": 18,
    "mosi": 23,
    "miso": 19,
    "output_amp_gain": 255,  # value between 0-255 controlling gain of output amplifier
}

get_param = lambda key: Pin(DEFAULT_SPI_PARAMS[key])
temp_spi_params = {key: get_param(key) for key in ["sck", "miso", "mosi"]}

spi = SPI(
    2,
    baudrate=10000000,
    polarity=0,
    phase=0,
    **temp_spi_params
)

data = bytearray([17, DEFAULT_SPI_PARAMS['output_amp_gain']])
spi.write(data)

ssid = 'TP-Link_AP_4C04'
password = '63525465'
connect_wifi(ssid, password)

###############################################################################

adc_sample = []

def sample_callback(*args, **kwargs):
    global adc_sample
    if len(adc_sample) > 256:
        del adc_sample[0]
        adc_sample.append(adc.read())
    else:
        adc_sample.append(adc.read())

sample_timer = Timer(0)
sample_timer.init(freq=256, callback=sample_callback)

# print(adc_sample)
# print("20000 ADC readings done after %u us." %(t2-t1))
# print("Mean time for each ADC reading = %15.13f us" % ((t2-t1)/20000.0))
# print("ADC reading = %15.13f ksamples/s" % (1000/((t2-t1)/20000.0)))

In [ ]:
while True:
    time.sleep(1)
    print(gc.mem_free())
    data = adc_sample
    toSend = {"raw_data":data}
    print(toSend)
    requests.JSONRequest("http://192.168.0.37:5001/collect", toSend)

In [12]:
requests.GETRequest("http://192.168.0.37:5001/save")

[leftinbuffer] ['659, 1840, 1936, 1776, 1599, 1638, 1817, 1914, 1791, 1610, 1595, 1759, 1893, 1792, 1607, 1569, 1727, 1873, 1808, 1616, 1552, 1695, 1861, 1830, 1646, 1552, 1677, 1852, 1849, 1639, 1552, 1682, 1843, 1872, 1699, 1569, 1648, 1839, 1901, 1737, 1593, 1649, 1840, 1929, 1778, 1618, 1651, 1831, 1939, 1808, 1637, 1639, 1808, 1935, 1820, 1600, 1607, 1792, 1909, 1823, 1627, 1571, 1719, 1873, 1821, 1633, 1552, 1687, 1857, 1840, 1657, 1555, 1671, 1847, 1855, 1674, 1547, 1641, 1826, 1870, 1697, 1557, 1630, 1845, 1878, 1727, 1577, 1617, 1802, 1904, 1765, 1605, 1619, 1801, 1923, 1803, 1632, 1627, 1801, 1937, 1843, 1659, 1622]}']
[leftinbuffer] ['24768']
[leftinbuffer] ["{'raw_data': [1871, 1795, 1616, 1598, 1744, 1901, 1745, 1649, 1598, 1744, 1914, 1885, 1699, 1611, 1741, 1916, 1911, 1727, 1611, 1715, 1899, 1920, 1741, 1601, 1680, 1859, 1909, 1718, 1577, 1643, 1824, 1899, 1741, 1580, 1599, 1774, 1879, 1744, 1574, 1580, 1749, 1877, 1771, 1589, 1565, 1724, 1872, 1795, 1615, 1563, 1717, 1

In [7]:
%serialconnect to --port="COM5" --baud=115200

Connecting to --port=COM5 --baud=115200 
Ready.


In [5]:
import machine
from machine import Pin
from machine import ADC, SPI
import time
from machine import Timer
import gc
from machine import freq
from lib.utils import connect_wifi
from lib.requests import MicroWebCli as requests
import ujson as json

freq(240000000)
adc = ADC(Pin(33))
adc.atten(machine.ADC.ATTN_11DB)
adc.width(machine.ADC.WIDTH_12BIT)
adcread = adc.read()

##################################################################################

DEFAULT_SPI_PARAMS = {
    "spi_num": 2,
    "sck": 18,
    "mosi": 23,
    "miso": 19,
    "output_amp_gain": 255,  # value between 0-255 controlling gain of output amplifier
}

get_param = lambda key: Pin(DEFAULT_SPI_PARAMS[key])
temp_spi_params = {key: get_param(key) for key in ["sck", "miso", "mosi"]}

spi = SPI(
    2,
    baudrate=10000000,
    polarity=0,
    phase=0,
    sck=Pin(18),
    mosi=Pin(23),
    miso=Pin(19),
)

# data = bytearray([17, DEFAULT_SPI_PARAMS['output_amp_gain']])
data = bytearray(DEFAULT_SPI_PARAMS['output_amp_gain'])
# data = bytearray(100)
spi.write(data)

ssid = 'TP-Link_AP_4C04'
password = '63525465'
connect_wifi(ssid, password)

###############################################################################

adc_sample = []

def sample_callback(*args, **kwargs):
    global adc_sample
    if len(adc_sample) > 256:
        del adc_sample[0]
        adc_sample.append(adc.read())
    else:
        adc_sample.append(adc.read())

sample_timer = Timer(0)
sample_timer.init(freq=256, callback=sample_callback)

for i in range(20):
    time.sleep(1)
    print(gc.mem_free())
    data = adc_sample
    toSend = {"raw_data":data}
    print(toSend)
    requests.JSONRequest("http://192.168.0.37:5001/collect", toSend)

requests.GETRequest("http://192.168.0.37:5001/save")

network config: ('192.168.0.84', '255.255.255.0', '192.168.0.1', '194.168.4.100')
71392
{'raw_data': [1787, 1665, 1623, 1739, 1861, 1831, 1690, 1638, 1753, 1867, 1851, 1708, 1633, 1718, 1853, 1858, 1717, 1628, 1695, 1827, 1856, 1717, 1616, 1664, 1799, 1850, 1718, 1601, 1635, 1776, 1842, 1732, 1603, 1621, 1763, 1840, 1749, 1614, 1613, 1748, 1851, 1781, 1644, 1616, 1742, 1850, 1797, 1655, 1613, 1728, 1854, 1823, 1680, 1618, 1720, 1850, 1842, 1702, 1626, 1712, 1865, 1862, 1716, 1629, 1697, 1836, 1870, 1729, 1621, 1671, 1808, 1861, 1732, 1613, 1643, 1779, 1853, 1744, 1611, 1621, 1756, 1840, 1749, 1615, 1609, 1741, 1836, 1775, 1634, 1607, 1733, 1846, 1799, 1655, 1609, 1725, 1851, 1826, 1682, 1616, 1715, 1854, 1849, 1705, 1622, 1708, 1846, 1866, 1727, 1634, 1701, 1841, 1876, 1733, 1633, 1680, 1817, 1872, 1751, 1625, 1657, 1793, 1866, 1761, 1622, 1636, 1770, 1856, 1769, 1630, 1619, 1748, 1850, 1778, 1637, 1606, 1726, 1840, 1790, 1638, 1600, 1709, 1831, 1806, 1661, 1595, 1696, 1828, 1824, 1681